In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
from datetime import datetime
import datetime as dt
import os

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
#Define the function to access fields within a layer --> provides a pandas dataframe of the data table

#Base tables for incorporation date and population
photo_base = gis.content.get("6d9d437bbe2d48bfa87b4d9fca41f80d")
community_base = gis.content.get("08d7d0a1178e4ee7826f7bfd2c5f5163")
incorp_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Community_Related/Community_Overview/MapServer/0')

#Find storymaps to edit
find_storymaps = gis.content.search("title: *Community Storymap", item_type="Web Mapping Application", max_items=500, outside_org=False)


In [ ]:
find_storymaps

In [ ]:
#Set up a log file to write errors to
#today = [dt.date.today()]
#log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Municipalities\Update Table of Contents"
#log_file = os.path.join(log_path, "Update Table of Contents_" + str(today[0]) + str(".txt"))
#out = open(log_file,'w')

for f in find_storymaps:
    try:
        if f.title != "Alaska DCRA Community Storymaps" and not f.title.startswith('Communit') and 'Communit' in str(f.title):
            print(f.title, f.id)
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            panel_toc = sections[0]
            community = f.title.replace(" Community Storymap", "")
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            if community.endswith('Borough'):
                community = community.replace(" Borough","")
            if community.endswith('(Unincorporated Community)'):
                community = community.replace(' (Unincorporated Community)','')
            query = "CommunityName = '" + str(community.replace("'","''")) + "' OR CommunityName = '" + str(community.replace("'","''")) + " CDP'"
            
            print_url = "https://dcced.maps.arcgis.com/apps/MapJournal/resources/tpl/viewer/print/print.html?appid=" + str(f.id)
            print_link = '<a href="'+ str(print_url)+ '" target="_blank"><img src="https://maps.commerce.alaska.gov/portal/sharing/rest/content/items/ae430ae18c0142d1aa49204915ba2b4b/data" width="278" align="left" alt="Print this storymap." height="56"></a>'
            survey_img="http://dcced.maps.arcgis.com/sharing/rest/content/items/0a7cc5234f3d491ab47c375b25d0527a/data"
            survey_link="https://arcg.is/14iHSD"
            embed_survey='<a href="' + str(survey_link) + '" target="_blank"><img align="right" alt="" height="56" src="' + str(survey_img) + '" /></a>'
            
            eng_pro_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Community_Related/Community_Overview/MapServer/2')
            eng_query = eng_pro_base.query(where=query)
            if str(eng_query.df) != 'Empty DataFrame\nColumns: []\nIndex: []':  
                eng_URL = eng_query.df.PronunciationAudio[0]
                eng_pron = eng_query.df.Pronounciation[0]
            else:
                eng_URL = '<i>Audio unavailable.</i>'
                eng_pron = '<i>Info unavailable.</i>'
            nat_place = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Community_Related/Community_Overview/MapServer/1')
            if str(nat_place.query(where=query,out_fields="*").df) != 'Empty DataFrame\nColumns: []\nIndex: []':
                NP_url = str(nat_place.query(where=query, out_fields="*").df.NativePlaceNameAudio[0])
                NP_url = '<audio controls src="' + str(NP_url) + '"></audio>'
                
                language = str(nat_place.query(where=query, out_fields="*").df.IndigenousLanguage[0])
                NP_name = str(nat_place.query(where=query, out_fields="*").df.NativeCommunityName[0])
                NP_translation = str(nat_place.query(where=query, out_fields="*").df.PlaceNameTranslation[0])
            else:
                NP_url = '<i>Audio unavailable.</i>'
                language = '<i> Info unavailable. </i>'                    
                NP_name = '<i> Info unavailable. </i>'
                NP_translation = '<i> Translation information is unavailable.</i>'

            #if community.endswith(" Borough"):
                #community=community.replace(" Borough", "")
            #THIS IS THE POPULATION INFORMATION AT THE TOP OF THE TABLE OF CONTENTS
            print('Finding population data...')
            pop_base = gis.content.get('70bcc629271744909b575cd1a525afa8')
            pop = pop_base.layers[0].query(where="CommunityName='"+str(community)+"' AND DataYear = 2018", out_fields = "*")
            if str(pop.df) != 'Empty DataFrame\nColumns: []\nIndex: []':
                current_pop = pop.df.Population[0]
                current_pop = str('{:,}'.format(current_pop)) + ' (Population Year: 2018)'
            else:
                current_pop = 'Current population unavailable.'

            incrp_type = str(pop_base.layers[0].query(where=query, out_fields="*").df.IncorporationType[0]).replace('CDP', 'Census-Designated Place')
            incrp_dt= None
            
            #TO DO, add an as-of date for the population (EZH, June 6 2019)
            if str(incrp_dt) == 'None':
                print("No incorporation date found for " + str(community.replace("'","''")))
                panel_toc["content"] = '<style type="text/css">.blue {\n     display: inline-block;\n     background-color: #2c7bb6;\n     border-color: #2c7bb6 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><em><strong><span style="font-size:12px">Please click &quot;Print This StoryMap&quot; for an ADA-compliant StoryMap.</span></strong></em></p><p>' + str(print_link) + str(embed_survey)  + '<p>&nbsp;</p><p><span style="font-weight: bold;">Current Population: ' + str(current_pop) + '</span></p><p><span style="font-size:16px">Incorporation Type: ' + str(incrp_type) + '</span></p><p>&nbsp;</p><p><b>Native Community Name: ' + str(NP_name) + '</b> (Language: ' + str(language) + ')</p><p><span style="font-size:16px">Native Community Name Translation: ' + str(NP_translation) + '</span></p><p><b>English Pronunciation: </b>' + str(eng_pron) + '</p><p>&nbsp;</p><p><table border=1><tr height="50px"><td width="50%"><i>Native Place Name Pronunciation: </i></td><td>' + str(NP_url) + '</td></tr><tr height="50px"><td width="50%"><i>English Pronunciation: </i></td><td><audio controls src="' + str(eng_URL) + '"></audio></td></tr></table></p><p>&nbsp;</p><p><span style=\"font-size:28px\">Table of Contents</span><a data-storymaps=\"MJ-ACTION-1524782969520\" data-storymaps-type=\"navigate\"> </a></p>\n\n<p>&nbsp;</p>\n\n<p><a data-storymaps=\"MJ-ACTION-1524782969520\" data-storymaps-type=\"navigate\">Culture and History</a></p>\n     <p><a data-storymaps=\"MJ-ACTION-1537213657245\" data-storymaps-type=\"navigate\">Geography and Climate</a></p>\n     <p><span style=\"font-size:20px\"><a class=\"peach\" data-storymaps=\"MJ-ACTION-1524783369112\" data-storymaps-type=\"navigate\">Demographics</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1524784937326\" data-storymaps-type=\"navigate\">Transportation</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1524786044096\" data-storymaps-type=\"navigate\">Education</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1537978181320\" data-storymaps-type=\"navigate\">Municipal Officials</a></span></p><p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1532548226455\" data-storymaps-type=\"navigate\">Economy</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1537213811429\" data-storymaps-type=\"navigate\">Business Licenses</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1532636740461\" data-storymaps-type=\"navigate\">General Services</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1551995653432\" data-storymaps-type=\"navigate\">Utilities</a></span></p>\n<p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1532636735244\" data-storymaps-type=\"navigate\">ANCSA&nbsp;</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1532636729718\" data-storymaps-type=\"navigate\">Community Contacts</a></span></p>\n     <p>&nbsp;</p>\n     <p><span style=\"font-size:14px\"><em>As you explore this storymap, you can click directly on map symbols and zoom around the map at any time to get more information about each item represented.</em></span></p>'
                
            else:
                incrp_date = str(incrp_dt)
                panel_toc["content"] = '<style type="text/css">.blue {\n     display: inline-block;\n     background-color: #2c7bb6;\n     border-color: #2c7bb6 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><em><strong><span style="font-size:12px">Please click &quot;Print This StoryMap&quot; for an ADA-compliant StoryMap.</span></strong></em></p><p>'+ str(print_link) + str(embed_survey)  + '<p>&nbsp;</p><p><span style="font-weight: bold;">Current Population: ' + str(current_pop) + '</span></p><p>&nbsp;</p><p><span style="font-size:16px">Incorporation Date: ' + str(incrp_date) + '</span></p><p>&nbsp;</p><p><span style="font-size:16px">Incorporation Type: ' + str(incrp_type) + '</span></p><p>&nbsp;</p>\n<p><b>Native Community Name: ' + str(NP_name) + '</b> (Language: ' + str(language) + ')</p><p><span style="font-size:16px">Native Community Name Translation: ' + str(NP_translation) + '</span></p><p><b>English Pronunciation: </b>' + str(eng_pron) + '</p><p>&nbsp;</p><p><table border=1><tr height="50px"><td><i>Native Place Name Pronunciation: </i></td><td>' + str(NP_url) +'</td></tr><tr height="50px"><td width="50%"><i>English Pronunciation: </i></td><td><audio controls src="' + str(eng_URL)+ '"></audio></td></tr></table></p><p>&nbsp;</p><p><span style=\"font-size:28px\">Table of Contents</span><a data-storymaps=\"MJ-ACTION-1524782969520\" data-storymaps-type=\"navigate\"> </a></p>\n\n<p>&nbsp;</p>\n\n<p><a data-storymaps=\"MJ-ACTION-1524782969520\" data-storymaps-type=\"navigate\">Culture and History</a></p>\n     <p><a data-storymaps=\"MJ-ACTION-1537213657245\" data-storymaps-type=\"navigate\">Geography and Climate</a></p>\n     <p><span style=\"font-size:20px\"><a class=\"peach\" data-storymaps=\"MJ-ACTION-1524783369112\" data-storymaps-type=\"navigate\">Demographics</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1524784937326\" data-storymaps-type=\"navigate\">Transportation</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1524786044096\" data-storymaps-type=\"navigate\">Education</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1537978181320\" data-storymaps-type=\"navigate\">Municipal Officials</a></span></p><p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1532548226455\" data-storymaps-type=\"navigate\">Economy</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1537213811429\" data-storymaps-type=\"navigate\">Business Licenses</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1532636740461\" data-storymaps-type=\"navigate\">General Services</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1551995653432\" data-storymaps-type=\"navigate\">Utilities</a></span></p>\n<p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1532636735244\" data-storymaps-type=\"navigate\">ANCSA&nbsp;</a></span></p>\n     <p><span style=\"font-size:20px\"><a data-storymaps=\"MJ-ACTION-1532636729718\" data-storymaps-type=\"navigate\">Community Contacts</a></span></p>\n     <p>&nbsp;</p>\n     <p><span style=\"font-size:14px\"><em>As you explore this storymap, you can click directly on map symbols and zoom around the map at any time to get more information about each item represented.</em></span></p>'
            
            panel_toc["contentActions"]  = [
                {
                            "id": "MJ-ACTION-1524782969520",
                            "type": "navigate",
                            "index": 1
                },
                {
                            "id": "MJ-ACTION-1524783369112",
                            "type": "navigate",
                            "index": 3
                },
                {
                            "id": "MJ-ACTION-1524784937326",
                            "type": "navigate",
                            "index": 4
                },
                {
                            "id": "MJ-ACTION-1524786044096",
                            "type": "navigate",
                            "index": 5
                },
                {
                            "id": "MJ-ACTION-1532548226455",
                            "type": "navigate",
                            "index": 7
                },
                {
                            "id": "MJ-ACTION-1532636729718",
                            "type": "navigate",
                            "index": 11
                },
                {
                            "id": "MJ-ACTION-1532636735244",
                            "type": "navigate",
                            "index": 10
                },
                {
                            "id": "MJ-ACTION-1532636740461",
                            "type": "navigate",
                            "index": 8
                },
                {
                            "id": "MJ-ACTION-1537213657245",
                            "type": "navigate",
                            "index": 2
                },
                {
                            "id": "MJ-ACTION-1537213811429",
                            "type": "navigate",
                            "index": 12
                },
                {
                            "id": "MJ-ACTION-1537817359227-85350",
                            "type": "navigate",
                            "index": 11
                },
                {
                            "id": "MJ-ACTION-1537978181320",
                            "type": "navigate",
                            "index": 6
                },
                {
                            "id": "MJ-ACTION-1551995653432",
                            "type": "navigate",
                            "index": 9
                }
            ]
                
            csmap.save()
            print(f.title + " storymap Table of Contents panel updated.***")

    except Exception as e:
        print("-------ISSUE WITH " + str(f.title) + "-------- Exception: " + str(e))
#        out.write("Table of Contents panel for " + str(f.title) +  " not properly updated. See storymap for more info.")
#        out.write('\n\n')
        print("Community written to error logfile.")
        continue
#out.close()